In [64]:
%load_ext autoreload
%autoreload 2
import manenv
from manenv.utils.product_utils import *
from manenv.core.world import World
from manenv.core.component import *
from manenv.render_wrapper import RenderWrapper
from manenv.core.product import *
from manenv.core.effector import *

import manenv.effectors as eff
import manenv.components as cmp


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
world = World((3, 3))
s = np.zeros((5, 5), dtype=np.int8)
s[1][2] = 2
s[1][1] = 2
s[2][1] = 1
s[3][1] = 1
product = Product(s)
world.place_component(make_vector(1, 1), cmp.Conveyor(
    np.array([
    [0, 0, 0],
    [1, 0, -1],
    [0, 0, 0]
])))
world.place_component(make_vector(2, 0), cmp.Conveyor(
    np.array([
    [0, -1, 0],
    [0, 0, 1],
    [0, 0, 0] 
])))
world.place_component(make_vector(2, 1), cmp.Outport())

grabber = eff.Grabber(make_vector(0, 2))
buddy = eff.Grabber(make_vector(1, 3))
welder = eff.Welder(make_vector(1, 2))
transformer = eff.Transformer(1, 2, make_vector(2,  4))
discard = eff.Acceptor(make_vector(1, 7))

world.place_component(make_vector(1, 2), cmp.Spawner(product))
assembler : cmp.Assembler = cmp.Assembler(make_vector(10, 10), [grabber, buddy, welder, discard])
world.place_component(make_vector(1, 0), assembler)
wrapper = RenderWrapper(world)

In [66]:
# wrapper.reset()
# assembler.place_in_inventory(product.copy())
# assembler.place_in_inventory(product.copy())
# world.update()

# grabber.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
# grabber.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
# buddy.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
# world.update()

# grabber.set_action(eff.GrabberActions.RELEASE.value)
# buddy.set_action(eff.GrabberActions.RELEASE.value)
# world.update()

# welder.set_action(eff.WelderActions.WELD_EAST.value)
# world.update()
# world.update()

# grabber.set_action(eff.GrabberActions.GRAB.value)
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
# world.update()

# grabber.set_action(eff.GrabberActions.RELEASE.value)
# world.update()

# transformer.set_action(eff.TransformerActions.TRANSFORM.value)
# world.update()
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
# world.update()
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
# world.update()


In [67]:
# wrapper.render()

# Gym Environment Test

In [68]:
from manenv.gym_wrapper import MARLFactoryEnvironment

In [69]:
# world.reset()
environment = MARLFactoryEnvironment(world)
renderer = RenderWrapper(environment._world)

In [70]:
from pettingzoo.test import parallel_api_test

In [71]:
def test():
    parallel_api_test(environment, num_cycles=1_000_000)

In [72]:
# test()

In [73]:
# import cProfile
# cProfile.run('test()', sort = 'time')

         168677371 function calls (168675527 primitive calls) in 68.995 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   185063   11.259    0.000   55.672    0.000 world.py:142(update)
 76882027    6.821    0.000    6.821    0.000 world.py:70(is_product_placed)
   370124    4.682    0.000   11.880    0.000 conveyor.py:56(update)
  1305511    2.489    0.000    2.489    0.000 _methods.py:90(_clip)
   185063    2.459    0.000   16.567    0.000 assembler.py:35(update)
        1    2.093    2.093   68.995   68.995 parallel_test.py:32(parallel_api_test)
   740252    1.537    0.000    1.537    0.000 {method 'integers' of 'numpy.random._generator.Generator' objects}
  2394432    1.236    0.000    1.351    0.000 world.py:61(place_product)
  6295702    1.207    0.000    2.383    0.000 {built-in method builtins.isinstance}
   852140    1.207    0.000    1.207    0.000 {method 'reduce' of 'numpy.ufunc' objects}
   185062    1.148    

KeyboardInterrupt: 

In [11]:
from manenv.solution.train_agents import train_loop
from manenv.solution.test_agents import test_agents

# Training Run

In [12]:
train_loop(environment, games=10000, seed=0)

Starting training on factory.
Using cuda device


KeyboardInterrupt: 

# Testing Run

In [ ]:
# test_agents(environment, games=1, seed=0)


Starting evaluation on factory (num_games=1)
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'6': 0, '7': 0, '8': 0, '10': 0}
{'

0.0